In [ ]:
%pip install \
  --force-reinstall \
  --no-deps \
  git+https://github.com/hemanth-asirvatham/GABRIEL-prerelease.git@main

In [ ]:
import os
import pandas as pd
import gabriel

from gabriel.utils.plot_utils import *

# verified org-level API key
os.environ['OPENAI_API_KEY'] = "..."

# assume counties_df is your input DataFrame with columns 'County' and 'FIPS'
PATH = os.path.expanduser("~/Documents/gabriel_tests")
save_dir = os.path.join(PATH, "criminal_legal_system_v1")
os.makedirs(save_dir, exist_ok=True)

# Load counties and duplicate location columns
counties_df = pd.read_excel("~/Downloads/us_counties_fips.xlsx")
counties_df["region"] = counties_df["County"].astype(str)  # region search hint
counties_df["country"] = "US"                              # country search hint

# County run

## Report generation

In [ ]:
# define topic and additional instructions
topics = ["criminal legal codes and policing rules"]

additional_instructions = """
For this specific county, produce a comprehensive report covering every significant statute, ordinance, regulation, administrative rule, and enforceable practice that shapes the criminal legal system.  
Identify all sources of authority—state law, county codes, municipal ordinances, agency regulations, police department policies, court rules, and prosecutorial guidelines—clearly distinguishing state-level vs. county/local rules.

Focus heavily on **crime-specific treatment and thresholds**.  
For each major category of offense—property crimes (theft, burglary, robbery, vandalism), violent crimes (assault, battery, homicide, domestic violence), sexual offenses, drug possession/distribution, weapons violations, traffic/crash-related offenses, fraud/white-collar crimes, stalking/harassment, and juvenile offenses—describe:

• Exact statutory definitions and key elements prosecutors must prove.  
• Monetary or quantitative cutoffs that separate misdemeanor from felony (e.g., dollar value for theft, quantity thresholds for drugs, injury severity for assault).  
• Aggravating or mitigating factors that trigger sentence enhancements or reduce penalties (e.g., use of a weapon, victim age, repeat offender status).  
• Mandatory minimums, presumptive sentencing ranges, and any alternative-sentencing programs (diversion, restorative justice, specialty courts).  
• Local prosecutorial practices or charging standards (e.g., policies on plea bargaining, declination thresholds for low-level property crime, treatment of domestic-violence cases).  
• Any notable departures from broader state norms—such as unusually high or low property-value thresholds, unique stalking definitions, special protections for certain victim groups, or distinctive treatment of juvenile defendants.

Also cover investigation and enforcement rules: arrest powers, stop-and-frisk standards, search/seizure limits, warrant practices, surveillance technologies (DNA databases, facial recognition, license-plate readers, predictive policing), and evidence-collection protocols.

Explain pre-trial procedures (bail/bond, arraignment timelines, discovery obligations, plea practices, diversion programs), trial and evidentiary standards (admissibility of digital evidence, hearsay exceptions, forensic requirements), sentencing guidelines, probation and parole rules, and any specialized courts (drug, mental-health, veterans).

Include civil-rights protections and oversight mechanisms: right to counsel, public-defender capacity, jury-selection rules, complaint procedures, civilian review boards, and recent reforms or controversies (bail reform, police-discipline changes, prosecutorial discretion debates).

Focus specifically on this specific county and how all of these rules and practices are implemented in this county.
Focus research on actual legal practice, prosecution, and policing in this county -- not generalities about criminal legal systems, but the actual rules that lawyers, prosecutors, and police follow in this county.
You should authentically represent the actual practice of law in this county -- not make up rules or apply generalities.
Use copious direct quotes of actual wording of statutes, ordinances, regulations, administrative rules, and enforceable practices; specific dollar amounts, sentencing years, important wording, everything that is important in criminal law in the county.
"""

# build prompts using the same template that the old Regional class used
template = gabriel.core.PromptTemplate.from_package("regional_analysis_prompt.jinja2")

# Build a DataFrame of prompts with identifiers and location columns
prompt_rows = []
for county in counties_df["County"].astype(str).dropna().unique():
    for topic in topics:
        prompt_text = template.render(
            region=county,
            topic=topic,
            additional_instructions=additional_instructions
        )
        prompt_rows.append({
            "prompt": prompt_text,
            "identifier": f"{county}|{topic}",
            "region": county,
            "country": "US"
        })
prompt_df = pd.DataFrame(prompt_rows)

# Call the new gabriel.whatever API using DataFrame input
responses = await gabriel.whatever(
    prompt_df,
    save_dir=save_dir,
    column_name="prompt",               # column containing the prompt text
    identifier_column="identifier",     # unique identifier for each prompt
    file_name="regional_responses.csv",
    model="gpt-5-mini",
    web_search=True,
    web_search_filters={"country": "country", "region": "region"},  # per-prompt filters
    n_parallels=350,
    search_context_size="medium",
    reasoning_effort="medium",
    use_dummy=False,
    reset_files=False
)

# Reconstruct the reports DataFrame with one row per county and topic
records = []
for _, row in responses.iterrows():
    region, topic = row["Identifier"].split("|", 1)
    text = row["Response"][0] if isinstance(row["Response"], list) else row["Response"]
    records.append({"region": region, "topic": topic, "report": text})

df_long = pd.DataFrame(records)
reports_df = df_long.pivot(index="region", columns="topic", values="report").reset_index()
reports_df.to_csv(os.path.join(save_dir, "regional_reports.csv"), index=False)

In [ ]:
reports_df

In [ ]:
# print out a random sample "criminal legal codes and policing rules" report
print(reports_df["criminal legal codes and policing rules"].iloc[9])



In [ ]:
import numpy as np

col = "criminal legal codes and policing rules"
wc = reports_df[col].dropna().astype(str).str.split().str.len()

shortest = int(wc.min())
median = float(np.median(wc))
longest = int(wc.max())

print(f"Shortest: {shortest}, Median: {median}, Longest: {longest}")

## Confirmation that reports done right

In [ ]:
do_classify = True  # toggle to skip this step
col = "criminal legal codes and policing rules"
if do_classify:
    wc = reports_df[col].fillna("").astype(str).str.split().str.len()
    reports_df_long = reports_df.loc[wc >= 500].copy()
    print(f"Dropped {len(reports_df) - len(reports_df_long)} reports below 500 words; keeping {len(reports_df_long)}")
    # Define a reasonable default label: flag reports that are rich in detail
    labels = {
        "report has enough detail": (
            "The report contains specific examples/direct evidence, is long and very thorough, and does not mention a complete lack of sources or being unable to find any information; report is mostly content, not mostly reflections on the inability to find information."
        )
    }
    classify_results = await gabriel.classify(
        reports_df_long,
        column_name=col,
        labels=labels,
        save_dir=os.path.join(save_dir, "classify"),
        model="gpt-5-mini",
        n_runs=1,
        reset_files=False,
    )
    # filter rows where the report passes
    passed = classify_results[classify_results["report has enough detail"] == True]
    frac_pass = len(passed) / len(classify_results)
    print(f"{len(passed)} of {len(classify_results)} reports passed ({frac_pass:.1%})")

## Evaluating attributes on reports

In [ ]:
# call gabriel.rank with your elo_attributes dictionary and desired hyperparameters
attributes = {
    "high felony-theft threshold":
        "How high the dollar cutoff is for theft to become a felony (higher value → more lenient toward property crime).",

    "low drug-possession threshold":
        "How small a quantity of controlled substances is needed for a felony charge (lower quantity → harsher).",

    "harsh sentencing for drug users":
        "Typical severity of penalties for simple possession or use compared to state norms (higher score → more severe).",

    "low bar for stalking and harassment":
        "Breadth of the statutory definition and ease of proving stalking/harassment (higher score → easier to prosecute).",

    "restrictive police engagement rules":
        "Degree of limits on police use of force, stop-and-frisk, or pursuit policies (higher score → more restrictive).",

    "aggressive surveillance allowance":
        "Extent of permitted use of facial recognition, license-plate readers, predictive policing, or DNA databases (higher score → more permissive).",

    "generous bail release":
        "Likelihood that defendants can obtain pre-trial release with low cash or non-monetary conditions (higher score → easier release).",

    "prosecutor charging aggressiveness":
        "Tendency to file the highest possible charges and enhancements (higher score → more aggressive).",

    "habitual-offender enhancements":
        "Prevalence and severity of repeat-offender or three-strikes statutes (higher score → harsher).",

    "lenient juvenile treatment":
        "Flexibility and diversion options for juveniles, including ease of transfer to adult court (higher score → more lenient toward youth).",

    "mandatory minimum prevalence":
        "Frequency of crimes carrying mandatory minimum prison terms (higher score → more mandatory minimums).",

    "plea-bargain dependence":
        "Extent to which convictions rely on plea deals rather than full trials (higher score → heavier reliance).",

    "evidence admissibility strictness":
        "Strictness of evidentiary standards (higher score → stricter; harder for prosecution).",

    "alternative sentencing availability":
        "Breadth of diversion, restorative justice, or specialty-court programs (higher score → more alternatives).",

    "sentencing length for violent crimes":
        "Typical prison terms for serious violent offenses compared with state averages (higher score → longer).",

    "property forfeiture friendliness":
        "Ease with which law enforcement can seize assets through civil or criminal forfeiture (higher score → more permissive)."
}

results = await gabriel.rate(
    passed,
    column_name="criminal legal codes and policing rules",
    attributes=attributes,
    save_dir=os.path.join(save_dir, "rate"),
    model="gpt-5",
)
"""
results = await gabriel.rank(
    passed,
    column_name="county visual character through the lens of locals",
    attributes=attributes,
    save_dir=os.path.join(save_dir, "rank"),
    model="gpt-5-mini",
    n_rounds=5,
    matches_per_round=6,
)
"""

In [ ]:
extract_attributes = {
    "felony theft threshold (USD)":
        "Dollar value at or above which theft is charged as a felony. Number only, USD.",

    "felony property-damage threshold (USD)":
        "Dollar amount of damage that elevates criminal mischief/property destruction to a felony. Number only, USD.",

    "lowest felony drug-possession threshold (grams)":
        "Smallest weight of a Schedule I/II substance that triggers felony possession. Number only, grams, no unit in output.",

    "statutory max jail for a misdemeanor (days)":
        "Maximum jail term authorized for a single misdemeanor count. Number only, days, no unit in output.",

    "maximum misdemeanor fine (USD)":
        "Highest monetary fine allowed for a single misdemeanor count. Number only, USD.",

    "violent-felony minimum time served before parole (%)":
        "Required fraction of a violent-felony sentence that must be served before parole eligibility. Number only, percent, no symbol in output.",

    "statute of limitations for felony theft (years)":
        "Limitations period for prosecuting felony theft. Number only, years, no unit in output.",

    "police stop documentation time limit (hours)":
        "Time within which officers must file a written report after a stop, search, or use-of-force incident. Number only, hours, no unit in output."
}


results = await gabriel.extract(
    results,
    column_name="criminal legal codes and policing rules",
    attributes=extract_attributes,
    save_dir=os.path.join(save_dir, "extract"),
    model="gpt-5",
)

# Making maps

In [ ]:
results = results.merge(counties_df[["County", "FIPS"]], left_on="region", right_on="County", how="left")
results = results.drop(columns=["region"])

In [ ]:
# produce choropleth maps for every attribute using the new MapMaker
maker = gabriel.utils.mapmaker.MapMaker(
    results,
    fips_col="FIPS",
    state_col=None,         # provide a state abbreviation column name if you have one
    country_col=None,       # provide an ISO-3 column name if you have world codes
    save_dir=save_dir,
    z_score=False,
    map_type="county",
)
# generate county (and optionally state/country) maps for each attribute
maker.make_maps(list(attributes.keys()))

In [ ]:
results

# Plots

In [ ]:
save_dir = os.path.join(PATH, "criminal_legal_system_v1")
results.to_csv(os.path.join(save_dir, "final_rate_results.csv"), index=False)

In [ ]:
import os, re, textwrap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.stats import sem, t
from tabulate import tabulate

# ------------------------------------------------------------- #
# 1.  FIPS helpers
# ------------------------------------------------------------- #
def _fips_from_parts(df: pd.DataFrame) -> pd.Series | None:
    if {"state", "county"}.issubset(df.columns):
        return (df["state"].astype(int).astype(str).str.zfill(2) +
                df["county"].astype(int).astype(str).str.zfill(3))
    return None

def _clean_fips(series: pd.Series) -> pd.Series:
    return (series.astype(str)
                  .str.replace(r"\.0$", "", regex=True)
                  .str.extract(r"(\d+)", expand=False)
                  .str.zfill(5))

# ------------------------------------------------------------- #
# 2.  CSV reader (handles long Attribute/Value format)
# ------------------------------------------------------------- #
def _read_any(path: str) -> pd.DataFrame:
    df = pd.read_csv(os.path.expanduser(path))

    if {"Attribute", "Value"}.issubset(df.columns):       # long → wide
        df = df[df["FIPS"].notna() & (df["FIPS"] != 0)]
        df = (df
              .pivot_table(index="FIPS",
                           columns="Attribute",
                           values="Value",
                           aggfunc="first")
              .reset_index())

    fips_col = next((c for c in df.columns if re.match(r"(?i).*fip", c)), None)
    if fips_col is None:
        fips_series = _fips_from_parts(df)
        if fips_series is None:
            raise ValueError(f"No FIPS or state+county combo in {path}")
    else:
        fips_series = _clean_fips(df[fips_col])
        df = df.drop(columns=[fips_col])

    df["FIPS"] = fips_series
    return df

# ------------------------------------------------------------- #
# 3.  Merge helper
# ------------------------------------------------------------- #
def load_and_merge(county_cov, social_cap, elo, *, how="inner") -> pd.DataFrame:
    df1, df2, df3 = map(_read_any, (county_cov, social_cap, elo))

    counts = pd.DataFrame({
        "file": ["cty_covariates", "social_capital", "county_elo"],
        "rows": [len(df1), len(df2), len(df3)],
        "unique FIPS": [df1["FIPS"].nunique(),
                        df2["FIPS"].nunique(),
                        df3["FIPS"].nunique()]
    })
    print("\nFile‑level counts:\n", counts.to_string(index=False), "\n")

    merged = (df1.merge(df2, on="FIPS", how=how, indicator="m1")
                 .merge(df3, on="FIPS", how=how, indicator="m2"))
    print(f"Rows after {how.upper()} merge: {len(merged):,}\n")

    if how == "outer":
        print("Overlap diagnostics:\n",
              merged[["m1", "m2"]].value_counts(dropna=False))
    return merged.drop(columns=["m1", "m2"])


health_df = pd.read_excel("~/Downloads/health_counties.xlsx")
health_df["FIPS"] = _clean_fips(health_df["FIPS"])

demographic_df = pd.read_csv("~/Downloads/county_complete.csv")
demographic_df["FIPS"] = _clean_fips(demographic_df["fips"])

vote_df = pd.read_csv("~/Downloads/county_2016_vote_share.csv")
vote_df["FIPS"] = _clean_fips(vote_df["FIPS"])

vote_shift_df = pd.read_csv("~/Downloads/county_dem_shift_2000_2016.csv")
vote_shift_df["FIPS"] = _clean_fips(vote_shift_df["FIPS"])

religion_df = pd.read_csv("~/Downloads/us_religion_census_major_rates.csv")
religion_df["FIPS"] = _clean_fips(religion_df["FIPS"])
religion_df = religion_df.rename(columns={"state": "religion_state"})

morals_df = pd.read_csv("~/Downloads/d_synth.csv")
morals_df["FIPS"] = _clean_fips(morals_df["FIPS"])
morals_df = morals_df.rename(columns={"state": "morals_state", "county": "morals_county"})

woke_df = pd.read_csv("~/Documents/runs/woke_county_v1/county_elo.csv")
woke_df["FIPS"] = _clean_fips(woke_df["FIPS"])

debt_df = pd.read_csv("~/Documents/runs/debt_county_v1/county_elo.csv")
debt_df["FIPS"] = _clean_fips(debt_df["FIPS"])

credit_score_df = pd.read_csv("~/Downloads/cty_cred_score_rP_gP_pall.csv")
credit_score_df["FIPS"] = _clean_fips(credit_score_df["cty"])

my_morals_df = pd.read_csv("~/Documents/runs/moral_county_v1/county_elo.csv")
my_morals_df["FIPS"] = _clean_fips(my_morals_df["FIPS"])
my_morals_df = my_morals_df.rename(columns={"County": "my_morals_county"})

enke_df = pd.read_stata("~/Downloads/county.dta")
enke_df["FIPS"] = _clean_fips(enke_df["county"])
enke_df = enke_df.rename(columns={"state": "enke_state", "county": "enke_county"})

legal_df = pd.read_csv("~/Documents/gabriel_tests/criminal_legal_system_v1/final_rate_results.csv")
legal_df["FIPS"] = _clean_fips(legal_df["FIPS"])
legal_df = legal_df.rename(columns={"County": "legal_county"})


In [ ]:
merged_df = load_and_merge(
    "~/Downloads/Unemployment2023.csv",
    "~/Downloads/social_capital_county.csv",
    "~/Documents/runs/us_history_county_v1/county_elo.csv"
)

# merge on FIPS with woke_df (only columns "wokeness in local high school history classes and curricula" and "FIPS")
merged_df = merged_df.merge(woke_df[["FIPS", "wokeness in local high school history classes and curricula"]], on="FIPS", how="left")
merged_df = merged_df.merge(health_df, on="FIPS", how="left")
merged_df = merged_df.merge(demographic_df, on="FIPS", how="left")
merged_df = merged_df.merge(vote_df, on="FIPS", how="left")
merged_df = merged_df.merge(vote_shift_df, on="FIPS", how="left")
merged_df = merged_df.merge(religion_df, on="FIPS", how="left")
merged_df = merged_df.merge(morals_df, on="FIPS", how="left")
merged_df = merged_df.merge(debt_df, on="FIPS", how="left")
merged_df = merged_df.merge(credit_score_df, on="FIPS", how="left")
merged_df = merged_df.merge(my_morals_df, on="FIPS", how="left")
merged_df = merged_df.merge(enke_df, on="FIPS", how="left")
merged_df = merged_df.merge(legal_df, on="FIPS", how="left")

# take the pop2018 column and do the log to get a log_pop2018 column
merged_df["log_pop2018"] = np.log(merged_df["pop2018"])
merged_df["log_density_2010"] = np.log(merged_df["density_2010"])
merged_df["median household income (in thousands)"] = merged_df["Median_Household_Income_2022"] / 1000


# Columns to blank out when the county is tiny
cols_to_null = [
    "ChristianAdherents", "ChristianRatePct", "EvangelicalRatePct",
    "MainlineRatePct", "BlackProtestantRatePct", "CatholicRatePct",
    "OrthodoxRatePct", "OtherChristianRatePct",
    "JewishAdherents", "JewishRatePct", "MuslimRatePct"
]

# Make sure POP2020 is numeric—silently coerce if someone snuck in strings
merged_df["POP2020"] = pd.to_numeric(merged_df["POP2020"], errors="coerce")

# Mask rows where population is under 5 000 and nuke the chosen columns
mask = merged_df["POP2020"] < 5_000
merged_df.loc[mask, cols_to_null] = np.nan


In [ ]:
pretty_names = {
    "child_ec_county": "childhood economic connectedness",
    "ec_county": "economic connectedness",
    "universalist_vs_communal_values": "net universalist values",
    "Average_credit_score_rP_gP_pall": "average credit score",
    "Median_Household_Income_2022": "median household income",
    "pop2018": "population",
    "log_pop2018": "log population",
    "log_density_2010": "log population density",
    "clustering_county": "friendship clustering",
    "volunteering_rate_county": "volunteering rate",
    "civic_organizations_county": "civic organization participation rate",
    "National Z-Score": "lost potential life years (z score)",
    "Preventable Hospitalization Rate": "preventable hospitalization rate",
    "Food Environment Index": "food environment index",
    "% With Access to Exercise Opportunities": "% with access to exercise opportunities",
    "% Completed High School": "% who have completed high school",
    "net_democrat": "net democrat vote share",
    "shift_dem_share": "shift in democrat vote share, 2000 to 2016",
    "black_2017": "percent black",
    "white_2017": "percent white",
    "asian_2017": "percent asian",
    "native_2017": "percent native american",
    "hispanic_2017": "percent hispanic",
    "bachelors_2017": "percent with bachelors degree",
    "unemployment_rate_2019": "unemployment rate",
    "households_speak_spanish_2019": "percent of households that speak spanish",
    "median_age_2019": "median age",
    "broadband_2017": "percent with broadband access",
    "ChristianRatePct": "percent christian",
    "EvangelicalRatePct": "percent evangelical",
    "BlackProtestantRatePct": "percent black protestant",
    "CatholicRatePct": "percent catholic",
    "harm_mean": "moral value of preventing suffering",
    "fairness_mean": "moral value of fairness",
    "loyalty_mean": "moral value of loyalty",
    "authority_mean": "moral value of respecting authority",
    "purity_mean": "moral value of spiritual purity",
    "Average Number of Mentally Unhealthy Days": "average number of mentally unhealthy days",
    "importance of playing sports in local high schools": "importance of playing high school sports",
    "prevalence of academic extracurricular activities in local high schools": "prevalence of academic extracurriculars",
    "importance of attending local high school football games to the community": "community importance of attending high school football games",
    "wokeness in local high school history classes and curricula": "wokeness in high school history classes"
}


In [ ]:
attributes.keys()

In [ ]:
regression_plot(
    merged_df,
    x=["net_democrat"],
    y=['high felony-theft threshold', 'low drug-possession threshold', 'harsh sentencing for drug users', 'low bar for stalking and harassment', 'restrictive police engagement rules', 'aggressive surveillance allowance', 'generous bail release', 'prosecutor charging aggressiveness', 'habitual-offender enhancements', 'lenient juvenile treatment', 'mandatory minimum prevalence', 'plea-bargain dependence', 'evidence admissibility strictness', 'alternative sentencing availability', 'sentencing length for violent crimes', 'property forfeiture friendliness', 'felony theft threshold (USD)'],
    controls=["median household income (in thousands)", "log_pop2018", "log_density_2010", "black_2017", "households_speak_spanish_2019", "median_age_2019", "bachelors_2017", "unemployment_rate_2019", "broadband_2017"],
    rename_map=pretty_names,
)